In [ ]:
# use the sierra db to find distinct patron records with holds in:
#     'b', -- Bib hold ready for pickup.
#     'j', -- Volume hold ready for pickup.
#     'i' -- Item hold ready for pickup.

# use the REST API to find the holds for that list of patrons above with a "pickupByDate"

# place the hold id into the sqlite db with the value for the "pickupByDate" field

!wget --quiet https://raw.githubusercontent.com/cincinnatilibrary/collection-analysis/master/misc/chpl_helper.py --output-document=chpl_helper.py

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, BigInteger, Numeric
import os
from getpass import getpass
import requests

import vars
import chpl_helper

sierra_engine = create_engine('postgresql://{}:{}@sierra-db.plch.net:1032/iii'.format(vars.pg_username, vars.pg_password))

# point this to the base URL for the Sierra REST API
base_url = 'https://classic.cincinnatilibrary.org:443/iii/sierra-api/v6/'

In [ ]:
# REMOVE and refresh the sqlite database file in the local directory
try:
    os.remove('items_pickup_by.db')
except:
    pass

os.close(os.open('items_pickup_by.db', os.O_CREAT))

engine = create_engine('sqlite:///items_pickup_by.db', echo=False)

In [ ]:
%%time
sql = """\
SELECT DISTINCT
--h.patron_record_id,
rm.record_num AS patron_record_num
FROM 
sierra_view."hold" AS h
JOIN sierra_view.record_metadata rm ON rm.id = h.patron_record_id 
WHERE 
h.status IN (
    'b', -- Bib hold ready for pickup
    'j', -- Volume hold ready for pickup.
    'i' -- Item hold ready for pickup.
)
"""

df = pd.read_sql(con=sierra_engine, sql=sql)
print(df.shape)

In [ ]:
# client_key = getpass('Enter the client KEY value: ')
# client_secret = getpass('Enter the client SECRET value: ')
auth_string = getpass('Enter the auth_string value: ')

# test our credentials
headers = chpl_helper.set_access_headers(base_url, auth_string=auth_string)
r = requests.get(base_url + 'info/token', headers=headers, verify=True)
print('access token expires in: {} seconds'.format(r.json()['expiresIn']))

In [ ]:
# get the "pickupByDate" values ...
# get /v6/patrons/{id}/holds 
# https://classic.cincinnatilibrary.org:443/iii/sierra-api/v6/patrons/1923269/holds?fields=pickupByDate

for i, patron in enumerate(df.itertuples()):
    r = requests.get(base_url + f'patrons/{patron[1]}/holds' , headers=headers, params={'fields': 'pickupByDate'})
    temp_df = pd.DataFrame([row for row in r.json()['entries'] if 'pickupByDate' in row])
    temp_df.to_sql(name='item_data', con=engine, if_exists='append')
